In [2]:
import tweepy
import pandas as pd
import time

# Twitter API credentials
API_KEY = 'RsSlIP3FwY4f1EIMc8bebHZEG'
API_SECRET_KEY = '673DJ5maWnRceyL6fF8BZLEayupqFBYWtLcTaBX3bBMZx7tkyq'
ACCESS_TOKEN = '1755730867886895104-HLSkSI1ZDxdsC6CFuxMxxE94RQLFOg'
ACCESS_TOKEN_SECRET = 'pgK7F9p2wRBDG0C3vqi1wgFFXWGHvPofLgkf79R1GNaJU'

BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAKiGxQEAAAAAczQM1DB71JI%2FqQty68FLJCuUylc%3Dpcmghg2Ho6Bnxwdcr2KG5mKOdBIwJtvNX90A5xyv1HcaLQDxdc'



In [3]:

client = tweepy.Client(bearer_token=BEARER_TOKEN)

In [6]:
# Cache for already processed users
user_cache = {}
location_cache = {}


try:
    progress_file = "progress.csv"
    df = pd.read_csv(progress_file)
    print("Resuming from last saved state.")
except FileNotFoundError:
    
    file_path = "../dataset/predicted_stances/updated_predicted_stances.csv"
    df = pd.read_csv(file_path)
    
    if 'latitude' not in df.columns:
        df['latitude'] = None
        df['longitude'] = None
        df['location_name'] = None

import os

def save_progress():
    """Update the DataFrame with cached values and save to a CSV file."""
    try:
        
        for user, location_data in user_cache.items():
            if location_data: 
                latitude, longitude, location_name = location_data
                df.loc[df['user'] == user, ['latitude', 'longitude', 'location_name']] = latitude, longitude, location_name

       
        temp_file = "progress_temp.csv"
        df.to_csv(temp_file, index=False)
        print(f"[DEBUG] Temporary progress saved to '{temp_file}'.")

       
        os.replace(temp_file, "progress.csv")
        print(f"[DEBUG] Temporary file renamed to 'progress.csv'. Progress saved.")
    except PermissionError as e:
        print(f"[ERROR] Permission error while saving progress: {e}")
    except Exception as e:
        print(f"[ERROR] Unexpected error while saving progress: {e}")


Resuming from last saved state.


In [7]:
import time

def get_user_location_cached(username):
    """Get user location from Twitter API with caching and rate limit handling."""
    if username in user_cache:
        if user_cache[username] is not None:
            print(f"[DEBUG] Returning cached location for user '{username}': {user_cache[username]}")
            return user_cache[username]
        else:
            print(f"[DEBUG] Cached value for user '{username}' is None. Retrying API call.")

    try:
        print(f"[DEBUG] Attempting to fetch user location for: {username}")
        time.sleep(1)  
        user = client.get_user(username=username, user_fields=['location'])
        profile_location = user.data.location if user and user.data else None
        print(f"[DEBUG] API response location for {username}: {profile_location}")
        user_cache[username] = profile_location
        return profile_location

    except tweepy.errors.TooManyRequests as e:
        
        reset_time = int(e.response.headers.get("x-rate-limit-reset", time.time() + 60))  
        wait_time = reset_time - int(time.time())
        print(f"[ERROR] Rate limit hit for Twitter API. Waiting for {wait_time} seconds...")
        time.sleep(max(wait_time, 0))  
        raise  

    except Exception as e:
        print(f"[ERROR] Unexpected error for user '{username}': {e}")
        user_cache[username] = None
        return None


In [ ]:
print(get_user_location_cached("RetirePedia"))

[DEBUG] Attempting to fetch user location for: RetirePedia
[ERROR] Rate limit hit for Twitter API. Waiting for 820 seconds...


In [47]:
import time

def wait_on_rate_limit():
    """Wait for a period to recover from the API rate limit."""
    wait_time = 2  
    print("Rate limit exceeded. Waiting...")
    time.sleep(wait_time) 


In [48]:
import requests

def geocode_location_cached(location_name):
    """Geocode location using GeoNames API with caching."""
    if location_name in location_cache:
        return location_cache[location_name]
    username = "alexandernys"  
    url = "http://api.geonames.org/searchJSON"
    params = {
        "q": location_name,
        "maxRows": 1,
        "username": username
    }
    try:
        
        time.sleep(1)  
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            if "geonames" in data and data["geonames"]:
                result = data["geonames"][0]
                lat, lng, loc_name = result["lat"], result["lng"], result["name"]
                location_cache[location_name] = (lat, lng, loc_name)
                return lat, lng, loc_name
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 429:
            print(f"Rate limit hit for GeoNames API when processing location: {location_name}")
            raise  
    except Exception as e:
        print(f"Error geocoding location with GeoNames API: {e}")
        return None, None, None

In [ ]:

try:
    total_rows = len(df)
    for index, row in df.iterrows():
        try:
            
            if pd.notna(row['latitude']):
                print(f"[INFO] Row {index + 1}/{total_rows}: Already processed. Skipping.")
                continue

            username = row['user']  
            if pd.notna(username):
                if username in user_cache:
                    print(f"[CACHE] Row {index + 1}/{total_rows}: Using cached location for user '{username}'.")
                    profile_location = user_cache[username]
                else:
                    print(f"[API] Row {index + 1}/{total_rows}: Fetching location from API for user '{username}'.")
                    profile_location = get_user_location_cached(username)
                    user_cache[username] = profile_location
                    print(f"[INFO] Sleeping for 2 seconds to avoid hitting API rate limits...")
                    time.sleep(2) 

                if profile_location:
                    if profile_location in location_cache:
                        print(f"[CACHE] Row {index + 1}/{total_rows}: Using cached geocode for location '{profile_location}'.")
                        lat, lng, loc_name = location_cache[profile_location]
                    else:
                        print(f"[API] Row {index + 1}/{total_rows}: Fetching geocode from API for location '{profile_location}'.")
                        lat, lng, loc_name = geocode_location_cached(profile_location)
                        location_cache[profile_location] = (lat, lng, loc_name)
                        print(f"[INFO] Sleeping for 2 seconds to avoid hitting API rate limits...")
                        time.sleep(2)  

                    
                    df.at[index, 'latitude'] = lat
                    df.at[index, 'longitude'] = lng
                    df.at[index, 'location_name'] = loc_name
                    print(f"[DEBUG] Updated row {index + 1}: {df.iloc[index]}")

            
            if index % 10 == 0:
                print(f"[DEBUG] Attempting to save progress at row {index + 1}/{total_rows}.")
                save_progress()
                print(f"[DEBUG] Save progress completed for row {index + 1}/{total_rows}.")

        except Exception as e:
            print(f"[ERROR] Unexpected error for user '{username}' at row {index + 1}/{total_rows}: {e}")
            user_cache[username] = None
            save_progress()
            continue

except KeyboardInterrupt:
    print("[INFO] Interrupted by user. Saving progress...")
    save_progress()

print("[INFO] Processing completed.")